In [5]:
import torch
import torchvision.transforms as transforms
from torchvision import models, io
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch import nn, optim
from sklearn.preprocessing import LabelEncoder

print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

labels_df = pd.read_csv('labels/labels.csv')

label_encoder = LabelEncoder()
labels_df['label'] = label_encoder.fit_transform(labels_df['label'])

base_dir = 'images/'

labels_df['image_path'] = labels_df['image_name'].apply(lambda x: os.path.join(base_dir, x))

train_df, test_df = train_test_split(labels_df, test_size=0.2, random_state=42)

# Transformaciones ajustadas para el entrenamiento
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),  # Ángulo reducido
    transforms.ColorJitter(contrast=1.2),  # Contraste reducido
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Transformaciones ajustadas para la evaluación
test_transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ColorJitter(contrast=1.2),  # Contraste reducido
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image_path']
        image = io.read_image(img_name).float() / 255.0
        label = self.df.iloc[idx]['label']
        label = torch.tensor(label, dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = CustomImageDataset(train_df, transform=train_transform)
test_dataset = CustomImageDataset(test_df, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Cargar el modelo InceptionV3 preentrenado
model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
model.aux_logits = True
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(label_encoder.classes_))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

train_losses = []
train_accuracy = []
val_accuracy = []

best_acc = 0.0

# Entrenamiento
for epoch in range(70):
    model.train()
    model.aux_logits = True
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs, aux_outputs = model(images)
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(train_loader))
    train_accuracy.append(100 * correct / total)

    # Evaluación
    model.eval()
    model.aux_logits = False
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.dim() == 1:
                outputs = outputs.unsqueeze(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_accuracy.append(val_acc)

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Train Accuracy: {100 * correct / total}%, Val Accuracy: {val_acc}%')

    # Checkpointing
    if val_acc > best_acc:
        best_acc = val_acc
        print("Actualización!")
        torch.save(model.state_dict(), 'best_model.pth')

PyTorch version: 2.1.2+cu121
GPU available: True


/home/rodrigo/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Loss: 4.757251569202968, Train Accuracy: 35.28089887640449%, Val Accuracy: 41.25560538116592%
Actualización!
Epoch 2, Loss: 4.470181005341666, Train Accuracy: 42.247191011235955%, Val Accuracy: 51.5695067264574%
Actualización!
Epoch 3, Loss: 4.190471759864262, Train Accuracy: 45.95505617977528%, Val Accuracy: 53.81165919282511%
Actualización!
Epoch 4, Loss: 3.896992496081761, Train Accuracy: 49.21348314606742%, Val Accuracy: 55.38116591928251%
Actualización!
Epoch 5, Loss: 3.653391284602029, Train Accuracy: 49.7752808988764%, Val Accuracy: 54.48430493273543%
Epoch 6, Loss: 3.4081968324525014, Train Accuracy: 50.2247191011236%, Val Accuracy: 56.2780269058296%
Actualización!
Epoch 7, Loss: 3.205998820917947, Train Accuracy: 52.92134831460674%, Val Accuracy: 57.399103139013455%
Actualización!
Epoch 8, Loss: 3.0441390701702664, Train Accuracy: 53.53932584269663%, Val Accuracy: 60.762331838565025%
Actualización!
Epoch 9, Loss: 2.8967044183186124, Train Accuracy: 53.146067415730336%

In [6]:
import torch
import pandas as pd
import os
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import inception_v3
from sklearn.preprocessing import LabelEncoder

model_path = 'best_model.pth'
model = inception_v3(weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(label_encoder.classes_))
model.load_state_dict(torch.load(model_path))
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_df = pd.read_csv('labels/test.csv')
base_dir = 'images/'
test_df['image_path'] = test_df['image_name'].apply(lambda x: os.path.join(base_dir, x))


class TestImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image_path']
        image = io.read_image(img_name).float() / 255.0
        if self.transform:
            image = self.transform(image)
        return image, self.df.iloc[idx]['image_name'], self.df.iloc[idx]['place'], self.df.iloc[idx]['date']


test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_dataset = TestImageDataset(test_df, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


results = []
with torch.no_grad():
    for images, names, places, dates in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        labels = [label_encoder.inverse_transform([pred.item()])[0] for pred in predicted]

        for name, place, date, label in zip(names, places, dates, labels):
            results.append([name, place, date, label])


results_df = pd.DataFrame(results, columns=['image_name', 'place', 'date', 'label'])
results_df.to_csv('resultado.csv', index=False)

/home/rodrigo/.local/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/rodrigo/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models w